# CSN4030 : Data Engineering
# Lab 1 : Travel Booking
# Group : Anuj Rajan Lalla  (B22AI061) , Abhinav Swami (B22AI003)

## Code for creation of tables (not records insertion)

In [50]:
import os
import mysql.connector

user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST")
database = os.getenv("MYSQL_DATABASE")

db_connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = db_connection.cursor()

# Creating the User Table
create_user_table = """
CREATE TABLE IF NOT EXISTS User (
    UserID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(100),
    Email VARCHAR(100),
    Password VARCHAR(100),
    ContactNumber VARCHAR(15),
    CreationTime TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cursor.execute(create_user_table)

# Creating the Flight Table with Availability
create_flight_table = """
CREATE TABLE IF NOT EXISTS Flight (
    FlightID INT AUTO_INCREMENT PRIMARY KEY,
    FlightNumber VARCHAR(10),
    DepartureCity VARCHAR(50),
    ArrivalCity VARCHAR(50),
    DepartureDate DATE,
    ArrivalDate DATE,
    Price DECIMAL(10, 2),
    Availability INT
);
"""
cursor.execute(create_flight_table)

# Creating the Hotel Table with Availability
create_hotel_table = """
CREATE TABLE IF NOT EXISTS Hotel (
    HotelID INT AUTO_INCREMENT PRIMARY KEY,
    HotelName VARCHAR(100),
    City VARCHAR(50),
    PricePerNight DECIMAL(10, 2),
    RoomType VARCHAR(50),
    Availability INT
);
"""
cursor.execute(create_hotel_table)

# Creating the Car Table with Availability
create_car_table = """
CREATE TABLE IF NOT EXISTS Car (
    CarID INT AUTO_INCREMENT PRIMARY KEY,
    CarType VARCHAR(50),
    City VARCHAR(50),
    PricePerDay DECIMAL(10, 2),
    RentalCompany VARCHAR(50),
    Availability INT
);
"""
cursor.execute(create_car_table)

# Creating the Booking Table with BookingHash
create_booking_table = """
CREATE TABLE IF NOT EXISTS Booking (
    BookingID INT AUTO_INCREMENT PRIMARY KEY,
    UserID INT,
    BookingDate DATE,
    TotalAmount DECIMAL(10, 2),
    BookingCreationTime TIME,
    BookingHash VARCHAR(255),  
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);
"""
cursor.execute(create_booking_table)

# Creating the Payment Table
create_payment_table = """
CREATE TABLE IF NOT EXISTS Payment (
    PaymentID INT AUTO_INCREMENT PRIMARY KEY,
    BookingID INT,
    PaymentDate DATE,
    Amount DECIMAL(10, 2),
    PaymentMethod VARCHAR(50),
    FOREIGN KEY (BookingID) REFERENCES Booking(BookingID) ON DELETE CASCADE
);
"""
cursor.execute(create_payment_table)

# Creating the BookingFlight Linking Table
create_booking_flight_table = """
CREATE TABLE IF NOT EXISTS BookingFlight (
    BookingID INT,
    FlightID INT,
    FOREIGN KEY (BookingID) REFERENCES Booking(BookingID) ON DELETE CASCADE,
    FOREIGN KEY (FlightID) REFERENCES Flight(FlightID),
    PRIMARY KEY (BookingID, FlightID)
);
"""
cursor.execute(create_booking_flight_table)

# Creating the BookingHotel Linking Table
create_booking_hotel_table = """
CREATE TABLE IF NOT EXISTS BookingHotel (
    BookingID INT,
    HotelID INT,
    FOREIGN KEY (BookingID) REFERENCES Booking(BookingID) ON DELETE CASCADE,
    FOREIGN KEY (HotelID) REFERENCES Hotel(HotelID),
    PRIMARY KEY (BookingID, HotelID)
);
"""
cursor.execute(create_booking_hotel_table)

# Creating the BookingCar Linking Table
create_booking_car_table = """
CREATE TABLE IF NOT EXISTS BookingCar (
    BookingID INT,
    CarID INT,
    FOREIGN KEY (BookingID) REFERENCES Booking(BookingID) ON DELETE CASCADE,
    FOREIGN KEY (CarID) REFERENCES Car(CarID),
    PRIMARY KEY (BookingID, CarID)
);
"""
cursor.execute(create_booking_car_table)

db_connection.commit()


## Defining cutsom hash function using roll numbers

In [51]:
def custom_hash(booking_id):
    # Using the common substring in the roll numbers
    prefix = "B22AI"
    combined = prefix + str(booking_id)
    hash_value = hash(combined)
    
    # Limit the hash value to a certain range, for example, 1000 buckets
    return hash_value % 1000


## Insertion of dummy records

In [52]:
# Insert Dummy Data into User Table with manual timestamps
import datetime

insert_users_with_time = """
INSERT INTO User (Name, Email, Password, ContactNumber,  CreationTime)
VALUES (%s, %s, %s, %s,  %s);
"""
user_data_with_time = [
    ('John Doe', 'john@example.com', 'password123', '1234567890',  '2024-08-14 15:30:00'),
    ('Jane Smith', 'jane@example.com', 'password456', '0987654321',  '2024-08-15 18:00:00'),
    ('Alice Brown', 'alice@example.com', 'password789', '1122334455', '2024-08-15 19:30:00'),
    ('Bob Green', 'bob@example.com', 'passwordabc', '6677889900',  '2024-08-16 10:00:00'),
    ('Eve White', 'eve@example.com', 'passworddef', '5566778899',  '2024-08-16 12:00:00')
]
cursor.executemany(insert_users_with_time, user_data_with_time)
db_connection.commit()

# Insert Dummy Data into Flight Table with Availability
insert_flights = """
INSERT INTO Flight (FlightNumber, DepartureCity, ArrivalCity, DepartureDate, ArrivalDate, Price, Availability)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""
flight_data = [
    ('AI101', 'New York', 'London', '2024-08-10', '2024-08-11', 800.00, 100),
    ('BA202', 'London', 'Paris', '2024-08-12', '2024-08-12', 150.00, 50),
    ('QR303', 'Doha', 'Tokyo', '2024-08-14', '2024-08-15', 1200.00, 200),
    ('EK404', 'Dubai', 'Singapore', '2024-08-16', '2024-08-16', 1000.00, 150),
    ('SQ505', 'Singapore', 'Sydney', '2024-08-18', '2024-08-19', 900.00, 120)
]
cursor.executemany(insert_flights, flight_data)
db_connection.commit()

# Insert Dummy Data into Hotel Table with Availability
insert_hotels = """
INSERT INTO Hotel (HotelName, City, PricePerNight, RoomType, Availability)
VALUES (%s, %s, %s, %s, %s);
"""
hotel_data = [
    ('Hilton', 'London', 200.00, 'Deluxe', 10),
    ('Marriott', 'Paris', 250.00, 'Suite', 5),
    ('Ritz', 'Tokyo', 300.00, 'Luxury', 8),
    ('Sheraton', 'Singapore', 180.00, 'Standard', 20),
    ('Intercontinental', 'Sydney', 220.00, 'Ocean View', 15)
]
cursor.executemany(insert_hotels, hotel_data)
db_connection.commit()

# Insert Dummy Data into Car Table with Availability
insert_cars = """
INSERT INTO Car (CarType, City, PricePerDay, RentalCompany, Availability)
VALUES (%s, %s, %s, %s, %s);
"""
car_data = [
    ('SUV', 'London', 100.00, 'Hertz', 5),
    ('Sedan', 'Paris', 80.00, 'Avis', 8),
    ('Luxury', 'Tokyo', 200.00, 'Enterprise', 2),
    ('Compact', 'Singapore', 70.00, 'Budget', 10),
    ('Convertible', 'Sydney', 150.00, 'Thrifty', 4)
]
cursor.executemany(insert_cars, car_data)
db_connection.commit()
# Function to insert booking with custom hash
def insert_booking_with_time(db_connection, booking_id, user_id, booking_date, total_amount):
    booking_hash = custom_hash(booking_id)
    current_time = datetime.datetime.now().time()  # Get the current time
    cursor = db_connection.cursor()
    insert_query = """
    INSERT INTO Booking (UserID, BookingDate, TotalAmount, BookingCreationTime, BookingHash)
    VALUES (%s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (user_id, booking_date, total_amount, current_time, booking_hash))
    db_connection.commit()
    cursor.close()
    
booking_data = [
    (1, '2024-08-01', 1000.00),  # Example: (UserID, BookingDate, TotalAmount)
    (2, '2024-08-05', 500.00),
    (3, '2024-08-10', 1500.00),
    (4, '2024-08-12', 800.00),
    (5, '2024-08-15', 1200.00)
]
# Insert Dummy Data into Booking Table
for i, booking in enumerate(booking_data, start=1):
    insert_booking_with_time(db_connection, i, booking[0], booking[1], booking[2])
# Insert Dummy Data into BookingFlight Table
insert_booking_flight = """
INSERT INTO BookingFlight (BookingID, FlightID)
VALUES (%s, %s);
"""
booking_flight_data = [
    (1, 1),  # BookingID 1 is linked to FlightID 1 (AI101)
    (2, 2),  # BookingID 2 is linked to FlightID 2 (BA202)
    (3, 3),  # BookingID 3 is linked to FlightID 3 (QR303)
    (4, 4),  # BookingID 4 is linked to FlightID 4 (EK404)
    (5, 5)   # BookingID 5 is linked to FlightID 5 (SQ505)
]
cursor.executemany(insert_booking_flight, booking_flight_data)
db_connection.commit()
# Insert Dummy Data into BookingHotel Table
insert_booking_hotel = """
INSERT INTO BookingHotel (BookingID, HotelID)
VALUES (%s, %s);
"""
booking_hotel_data = [
    (1, 1),  # BookingID 1 is linked to HotelID 1 (Hilton)
    (2, 2),  # BookingID 2 is linked to HotelID 2 (Marriott)
    (3, 3),  # BookingID 3 is linked to HotelID 3 (Ritz)
    (4, 4),  # BookingID 4 is linked to HotelID 4 (Sheraton)
    (5, 5)   # BookingID 5 is linked to HotelID 5 (Intercontinental)
]
cursor.executemany(insert_booking_hotel, booking_hotel_data)
db_connection.commit()
# Insert Dummy Data into BookingCar Table
insert_booking_car = """
INSERT INTO BookingCar (BookingID, CarID)
VALUES (%s, %s);
"""
booking_car_data = [
    (1, 1),  # BookingID 1 is linked to CarID 1 (SUV, Hertz)
    (2, 2),  # BookingID 2 is linked to CarID 2 (Sedan, Avis)
    (3, 3),  # BookingID 3 is linked to CarID 3 (Luxury, Enterprise)
    (4, 4),  # BookingID 4 is linked to CarID 4 (Compact, Budget)
    (5, 5)   # BookingID 5 is linked to CarID 5 (Convertible, Thrifty)
]
cursor.executemany(insert_booking_car, booking_car_data)
db_connection.commit()




## Applying clustering and secondary indexing

## MySql does not allow to create a 'clustering' index as such as it clusters data on primary key
## But keeping in mind the role of clustering indexing, I have created an index on BookingDate which will simplify ranged date query lookups

In [53]:
# Create an index on BookingDate (this will not cluster the data, but will optimize queries)
apply_index_on_booking_date = "CREATE INDEX idx_booking_date ON Booking(BookingDate);"
cursor.execute(apply_index_on_booking_date)

# Apply a Secondary Index on UserID
apply_secondary_index_on_bookinghash = "CREATE INDEX idx_booking_hash ON Booking(BookingHash);"
cursor.execute(apply_secondary_index_on_bookinghash)

db_connection.commit()


## Below are Sql queries for checking storage and execution time

For storage (This needs to be applied once before executing the indexing cell to check storage although there would be no changes as no 'clustering index is created'):
`SHOW TABLE STATUS LIKE 'Booking';`

For time: (For clustering index [Primary key in this case but can check for BookingDate too])
`EXPLAIN ANALYZE
 SELECT * FROM Booking
 FORCE INDEX (PRIMARY)
 WHERE BookingDate BETWEEN '2024-08-01' AND '2024-08-31'`

`FORCE INDEX (PRIMARY)` would be replaced by `FORCE INDEX (idx_booking_date / idx_booking_hash)`

In [54]:

# 1. Add Information About 5 New Users
insert_users_query = """
INSERT INTO User (Name, Email, Password, ContactNumber, CreationTime)
VALUES (%s, %s, %s, %s, %s);
"""

user_data_with_time = [
    ('Charlie Black', 'charlie@example.com', 'password123', '5551234567', '2024-08-17 10:30:00'),
    ('Dana White', 'dana@example.com', 'password456', '5552345678', '2024-08-17 11:00:00'),
    ('Edward Green', 'edward@example.com', 'password789', '5553456789', '2024-08-17 11:30:00'),
    ('Fiona Blue', 'fiona@example.com', 'password012', '5554567890', '2024-08-17 12:00:00'),
    ('George Brown', 'george@example.com', 'password345', '5555678901', '2024-08-17 12:30:00')
]

cursor.executemany(insert_users_query, user_data_with_time)
db_connection.commit()

# 2. Prepare a Report on All Bookings Made in August 2024
report_query = """
SELECT * FROM Booking
WHERE BookingDate BETWEEN '2024-08-01' AND '2024-08-31';
"""

cursor.execute(report_query)
august_bookings = cursor.fetchall()

# Process or print the result
print("Bookings made in August 2024:")
for booking in august_bookings:
    print(booking)


Bookings made in August 2024:
(1, 1, datetime.date(2024, 8, 1), Decimal('1000.00'), datetime.timedelta(seconds=41547), '237')
(2, 2, datetime.date(2024, 8, 5), Decimal('500.00'), datetime.timedelta(seconds=41547), '949')
(3, 3, datetime.date(2024, 8, 10), Decimal('1500.00'), datetime.timedelta(seconds=41547), '367')
(4, 4, datetime.date(2024, 8, 12), Decimal('800.00'), datetime.timedelta(seconds=41547), '298')
(5, 5, datetime.date(2024, 8, 15), Decimal('1200.00'), datetime.timedelta(seconds=41547), '874')


In [55]:
#3. Remove All User Profiles Made After 7 PM on August 15, 2024
delete_users_query = """
DELETE FROM User
WHERE CreationTime > '2024-08-15 19:00:00';
"""

cursor.execute(delete_users_query)
db_connection.commit()

cursor.close()
db_connection.close()